In [32]:
# Dependencies and imports
!pip install lorem-text

from lorem_text import lorem
from yfiles_jupyter_graphs import GraphWidget

In [177]:
# Connect to DB
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://89698250.databases.neo4j.io"
AUTH = ("neo4j", "nN4v9Y33RjQtEPzqgNAUUbKQj6Os9Hbs_AuGSY7s1DQ")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [186]:
# Clear the DB
records, summary, keys = driver.execute_query(
    "MATCH (n) DETACH DELETE (n);",
    database_="neo4j",
)

In [151]:
# Set unique constraints

queries = []

queries.append('CREATE CONSTRAINT s_name_unique IF NOT EXISTS FOR (n:Supplier) REQUIRE n.name IS UNIQUE;')
queries.append('CREATE CONSTRAINT s_sku_unique IF NOT EXISTS FOR (n:Assembly) REQUIRE n.sku IS UNIQUE;')
queries.append('CREATE CONSTRAINT b_sku_unique IF NOT EXISTS FOR (n:Product) REQUIRE n.sku IS UNIQUE;')
queries.append('CREATE CONSTRAINT p_sku_unique IF NOT EXISTS FOR (n:Part) REQUIRE n.sku IS UNIQUE;')
queries.append('CREATE CONSTRAINT d_sku_version_unique IF NOT EXISTS FOR (n:Description) REQUIRE (n.sku, n.version) IS UNIQUE;')
queries.append('CREATE CONSTRAINT s_sku_version_unique IF NOT EXISTS FOR (n:Specification) REQUIRE (n.sku, n.version) IS UNIQUE;')

# Execute Queries
for query in queries:
    records, summary, keys = driver.execute_query(
    query,
    database_="neo4j",
)


In [152]:
# Create the Suppliers

suppliers = [
    {'code': 'FR001', 'name': 'XYZ Co (self)', 'country': "FR"},
    {'code': 'FR002', 'name': 'Supplier #1', 'country': "FR"},
    {'code': 'CN001', 'name': 'Supplier #2', 'country': "CN"},
    {'code': 'US001', 'name': 'Supplier #3', 'country': "US"},
    {'code': 'UK001', 'name': 'Supplier #4', 'country': "UK"},
]

cypher_query = """
WITH $json as data
UNWIND data AS s 
MERGE (supplier:Supplier {code:s.code}) SET
	supplier.name = s.name, 
	supplier.country = s.country
"""

with driver.session() as session:
    result = session.run(cypher_query, json = suppliers)


print("Done")


In [153]:
# Create Parts and supplier relationships
queries = []

parts = [
    # Spokes
    {'sku': 'sp01', 'name': 'Spoke - STD', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 100},
            {'code': 'FR002', 'cost': 110},
            {'code': 'US001', 'cost': 120},
    ]},
    {'sku': 'sp02', 'name': 'Spoke - FR', 'uom': 'pc', 'suppliers': [
            {'code': 'FR002', 'cost': 115},
    ]},
    {'sku': 'sp03', 'name': 'Spoke - XS', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 90},
            {'code': 'FR002', 'cost': 100},
            {'code': 'US001', 'cost': 110},
    ]},

    # Hubs
    {'sku': 'hb01', 'name': 'Hub', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 60},
            {'code': 'US001', 'cost': 89},
    ]},
    {'sku': 'hb02', 'name': 'Hub - SM', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 65},
            {'code': 'FR002', 'cost': 90},
            {'code': 'US001', 'cost': 93},
    ]},
    {'sku': 'hb03', 'name': 'Hub - GF', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 40},
            {'code': 'FR002', 'cost': 70},
            {'code': 'US001', 'cost': 69},
    ]},
    {'sku': 'hb04', 'name': 'Hub - KL', 'uom': 'pc', 'suppliers': [
            {'code': 'FR002', 'cost': 140},
            {'code': 'US001', 'cost': 119},
    ]},

    # Rims
    {'sku': 'rm01', 'name': 'Rim - STD', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 600},
            {'code': 'US001', 'cost': 510},
    ]},
    {'sku': 'rm02', 'name': 'Rim - XT', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 620},
            {'code': 'US001', 'cost': 530},
    ]},
    {'sku': 'rm03', 'name': 'Rim - XR', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 590},
            {'code': 'US001', 'cost': 505},
    ]},

    # Tires
    {'sku': 'ty01', 'name': 'Tire - STD', 'uom': 'pc', 'suppliers': [
            {'code': 'FR002', 'cost': 568},
            {'code': 'CN001', 'cost': 320},
            {'code': 'US001', 'cost': 700},
    ]},
    {'sku': 'ty02', 'name': 'Tire - MB', 'uom': 'pc', 'suppliers': [
            {'code': 'FR002', 'cost': 560},
            {'code': 'US001', 'cost': 710},
    ]},
    {'sku': 'ty03', 'name': 'Tire - ST', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 325},
            {'code': 'US001', 'cost': 705},
    ]},

    # Valves
    {'sku': 'vlv01', 'name': 'Valve', 'uom': 'pc', 'suppliers': [
            {'code': 'FR002', 'cost': 42},
            {'code': 'CN001', 'cost': 25},
            {'code': 'US001', 'cost': 32},
    ]},
    {'sku': 'hbr01', 'name': 'Handlebar', 'uom': 'pc', 'suppliers': [
            {'code': 'FR002', 'cost': 234},
    ]},
    {'sku': 'hdg01', 'name': 'Handlebar grip', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 121},
            {'code': 'FR002', 'cost': 234},
            {'code': 'US001', 'cost': 210},
    ]},
    {'sku': 'fb01', 'name': 'Front brake', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 451},
            {'code': 'US001', 'cost': 612},
    ]},
    {'sku': 'fo01', 'name': 'Fork', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 213},
            {'code': 'FR001', 'cost': 250},
            {'code': 'FR002', 'cost': 300},
            {'code': 'US001', 'cost': 320},
    ]},
    {'sku': 'sh01', 'name': 'Shifter', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 221},
            {'code': 'US001', 'cost': 240},
    ]},
    {'sku': 's01', 'name': 'Saddle', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 111},
            {'code': 'UK001', 'cost': 125},
    ]},
    {'sku': 'se02', 'name': 'Seat post', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 222},
            {'code': 'US001', 'cost': 300},
    ]},

    # Transmission
    {'sku': 'fd01', 'name': 'Front Derailleur', 'uom': 'pc', 'suppliers': [
            {'code': 'US001', 'cost': 451},
    ]},

    {'sku': 'ch01', 'name': 'Chain', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 234},
    ]},
    {'sku': 'cr01', 'name': 'Chain Rings', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 60},
            {'code': 'FR002', 'cost': 60},
            {'code': 'US001', 'cost': 89},
    ]},
    {'sku': 'fr01', 'name': 'Frame', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 212},
            {'code': 'FR001', 'cost': 250},
            {'code': 'FR002', 'cost': 300},
            {'code': 'US001', 'cost': 320},
    ]},
    {'sku': 'rb01', 'name': 'Rear brakes', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 60},
            {'code': 'US001', 'cost': 89},
    ]},
    {'sku': 'cs01', 'name': 'Cog set', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 261},
            {'code': 'US001', 'cost': 211},
    ]},
    {'sku': 'rd01', 'name': 'Rear Derailluer', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 340},
            {'code': 'US001', 'cost': 432},
    ]},
    {'sku': 'pd01', 'name': 'Pedal', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 60},
            {'code': 'FR002', 'cost': 80},
            {'code': 'US001', 'cost': 62},
    ]},
    {'sku': 'ca01', 'name': 'Crank Arm', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 60},
            {'code': 'FR001', 'cost': 120},
            {'code': 'US001', 'cost': 89},
    ]},
    {'sku': 'drk_grn', 'name': 'Green', 'uom': 'm2', 'suppliers': [
            {'code': 'FR001', 'cost': 100},
    ]},
    {'sku': 'st01', 'name': 'Stem', 'uom': 'pc', 'suppliers': [
            {'code': 'FR001', 'cost': 400},
    ]},
    {'sku': 'hd01', 'name': 'Head', 'uom': 'pc', 'suppliers': [
            {'code': 'CN001', 'cost': 500},
            {'code': 'US001', 'cost': 550},
    ]},

]

for part in parts:
    sku = part['sku']
    name = part['name']
    uom = part['uom']
    
    query = f"CREATE (:Part {{sku: '{sku}', name: '{name}', uom: '{uom}'}})"
    queries.append(query)
    
#     # Part Description
#     query = f"CREATE (:Description {{sku: '{sku}', body: '{lorem.sentence()}', version: 'latest'}})"
#     queries.append(query)
#     query = f"MATCH (p:Part {{sku: '{sku}'}}), (d:Description {{sku: '{sku}'}}) CREATE (p)-[:IS_DESCRIBED_BY]->(d)"
#     queries.append(query)
# #     query = f"MATCH (p:Part {{sku: '{sku}'}}), (d:Description {{sku: '{sku}'}}) CREATE (d)-[:DESCRIBES]->(p)"
# #     queries.append(query)

#     # Part Specification
#     query = f"CREATE (:Specification {{sku: '{sku}', body: '{lorem.sentence()}', version: 'latest'}})"
#     queries.append(query)
#     query = f"MATCH (p:Part {{sku: '{sku}'}}), (d:Specification {{sku: '{sku}'}}) CREATE (p)-[:IS_SPECIFIED_BY]->(d)"
#     queries.append(query)
# #     query = f"MATCH (p:Part {{sku: '{sku}'}}), (d:Specification {{sku: '{sku}'}}) CREATE (d)-[:SPECIFIES]->(p)"
# #     queries.append(query)

    counter = 0
    for supplier in part['suppliers']:
        supplier_code = supplier['code']
        supplier_cost = supplier['cost']
        
	# Supplier
        part_supplier_query = f"MATCH (p:Part {{sku: '{sku}'}}), (s:Supplier {{code: '{supplier_code}'}}) CREATE (p)-[:IS_SOURCED_FROM {{cost: {supplier_cost}}}]->(s)"
        queries.append(part_supplier_query)
        # supplier_part_query = f"MATCH (p:Part {{sku: '{sku}'}}), (s:Supplier {{code: '{supplier_code}'}}) CREATE (s)-[:SUPPLIES {{cost: {supplier_cost}}}]->(p)"
        # queries.append(supplier_part_query)

	# Selected Supplier / Winning Supplier / Contracted Supplier
        if counter == 0:
            selected_supplier_query = f"MATCH (p:Part {{sku: '{sku}'}}), (s:Supplier {{code: '{supplier_code}'}}) CREATE (p)-[:SELECTED {{cost: {supplier_cost}}}]->(s)"
            queries.append(selected_supplier_query)

        counter +=1
        


# Print the queries
# for query in queries:
#     print(query)
    
# Execute Queries
for query in queries:
    records, summary, keys = driver.execute_query(
    query,
    database_="neo4j",
)



In [154]:
# Create Product(s) and relationships
queries = []

products = [
    {
        "sku": "ss01",
        "name": "SuperSport",
        "retail_price": 29999,
        "assemblies": [
            {'sku': 'co01', 'name': 'Color', 'parts': [
                {'sku':'drk_grn','qty':2, 'order': 1},
                ],'qty':1, 'order':99},
            {'sku': 'whl01', 'name': 'Wheel Assembly', 'parts': [
                {'sku':'sp01','qty':21, 'order': 2},
                {'sku':'hb01','qty':1, 'order': 1},
                {'sku':'rm01','qty':1, 'order': 3},
                {'sku':'ty01','qty':1, 'order': 5},
                {'sku':'vlv01','qty':1, 'order': 4},
                ],'qty':2, 'assemblies': [
                    {'sku': 'co01'}
                ], 'order':1, 'alternative_parts': [
                    {'sku': 'rm02'},
					{'sku': 'rm03'},
					{'sku': 'rm04'},
					{'sku': 'ty02'},
					{'sku': 'ty03'},
					{'sku': 'ty04'},
                ]},
            {'sku': 'fs01', 'name': 'Front Set', 'parts': [
                {'sku':'hbr01','qty':1, 'order': 1},
                {'sku':'hdg01','qty':2, 'order': 2},
                {'sku':'fb01','qty':1, 'order': 3},
                {'sku':'fo01','qty':1, 'order': 5},
                {'sku':'sh01','qty':1, 'order': 4},
                ],'qty':1, 'assemblies': [
                    {'sku': 'co01'}
                ], 'order':3},
            {'sku': 'sa01', 'name': 'Saddle Area', 'parts': [
                {'sku':'s01','qty':1, 'order': 2},
                {'sku':'se02','qty':1, 'order': 1},
                ],'qty':1, 'order':2},
            {'sku': 'tr01', 'name': 'Transmission', 'parts': [
                {'sku':'fd01','qty':1, 'order': 2},
                {'sku':'ch01','qty':1, 'order': 1},
                {'sku':'cr01','qty':1, 'order': 3},
                {'sku':'rb01','qty':1, 'order': 4},
                {'sku':'cs01','qty':1, 'order': 5},
                {'sku':'rd01','qty':1, 'order': 6},
                {'sku':'rca01','qty':2, 'order': 7},
                {'sku':'pd01','qty':2, 'order': 8},
                ],'qty':1, 'order':5},
            {'sku': 'fr01', 'name': 'Frame', 'parts': [
                {'sku':'fr01','qty':1, 'order': 1},
                {'sku':'st01','qty':1, 'order': 2},
                {'sku':'hd01','qty':1, 'order': 3},
                ],'qty':1, 'assemblies': [
                    {'sku': 'co01'}
                ], 'order':4},
        ],
    }
]

for product in products:
    sku = product['sku']
    name = product['name']
    retail_price = product['retail_price']

    # Create Product
    product_query = f"CREATE (:Product {{sku: '{sku}', name: '{name}', retail_price: {retail_price}}})"
    queries.append(product_query)
    
    # Product Description
    query = f"CREATE (:Description {{sku: '{sku}', body: '{lorem.sentence()}', version: 'latest'}})"
    queries.append(query)
    query = f"MATCH (p:Product {{sku: '{sku}'}}), (d:Description {{sku: '{sku}'}}) CREATE (p)-[:IS_DESCRIBED_BY]->(d)"
    queries.append(query)
    # query = f"MATCH (p:Product {{sku: '{sku}'}}), (d:Description {{sku: '{sku}'}}) CREATE (d)-[:DESCRIBES]->(p)"
    # queries.append(query)

    # Product Specification
    query = f"CREATE (:Specification {{sku: '{sku}', body: '{lorem.sentence()}', version: 'latest'}})"
    queries.append(query)
    query = f"MATCH (p:Product {{sku: '{sku}'}}), (d:Specification {{sku: '{sku}'}}) CREATE (p)-[:IS_SPECIFIED_BY]->(d)"
    queries.append(query)
    # query = f"MATCH (p:Product {{sku: '{sku}'}}), (d:Specification {{sku: '{sku}'}}) CREATE (d)-[:SPECIFIES]->(p)"
    # queries.append(query)
    
    for assembly in product['assemblies']:
        assembly_sku = assembly['sku']
        assembly_name = assembly['name']
        assembly_order = assembly['order']
        assembly_qty = assembly['qty']
        
        assembly_query = f"CREATE (:Assembly {{sku: '{assembly_sku}', name: '{assembly_name}', order: {assembly_order}}})"
        queries.append(assembly_query)
        
        # Product Assembly
        # assembly_part_query = f"MATCH (a:Assembly {{sku: '{assembly_sku}'}}), (p:Product {{sku: '{sku}'}}) CREATE (a)-[:BELONGS_TO {{qty: {qty}, order: {order}}}]->(p)"
        # queries.append(assembly_part_query)
        part_assembly_query = f"MATCH (a:Assembly {{sku: '{assembly_sku}'}}), (p:Product {{sku: '{sku}'}}) CREATE (p)-[:HAS {{order: {assembly_order}, qty: {assembly_qty}}}]->(a)"
        queries.append(part_assembly_query)
        
        if 'parts' in assembly:
            for part in assembly['parts']:
                part_sku = part['sku']
                qty = part['qty']
                order = part['order']
                
                # Part Assembly
                assembly_part_query = f"MATCH (a:Assembly {{sku: '{assembly_sku}'}}), (p:Part {{sku: '{part_sku}'}}) CREATE (a)-[:HAS {{qty: {qty}, order: {order}}}]->(p)"
                queries.append(assembly_part_query)
                # part_assembly_query = f"MATCH (a:Assembly {{sku: '{assembly_sku}'}}), (p:Part {{sku: '{part_sku}'}}) CREATE (p)-[:BELONGS_TO {{qty: {qty}, order: {order}}}]->(a)"
                # queries.append(part_assembly_query)
                
        if 'assemblies' in assembly:
            for a in assembly['assemblies']:
                a_sku = a['sku']
                        
                assembly_part_query = f"MATCH (a:Assembly {{sku: '{assembly_sku}'}}), (b:Assembly {{sku: '{a_sku}'}}) CREATE (a)-[:HAS]->(b)"
                queries.append(assembly_part_query)

        if 'alternative_parts' in assembly:
            # print("here")
            for a in assembly['alternative_parts']:
                # print(a)
                alternative_part_sku = a['sku']
                        
                alternative_part_query = f"MATCH (a:Assembly {{sku: '{assembly_sku}'}}), (p:Part {{sku: '{alternative_part_sku}'}}) CREATE (a)-[:COULD_HAVE]->(p)"
                queries.append(alternative_part_query)



# # Print the queries
# for query in queries:
#     print(query)

# Execute Queries
for query in queries:
    records, summary, keys = driver.execute_query(
    query,
    database_="neo4j",
)




In [155]:
# Query DB for all nodes and all relationships

with driver.session(database="neo4j") as session:
  graph = session.run("MATCH p=()-[]->() RETURN p;").graph()

# graph


In [156]:
# custom graph functions

def custom_node_color_mapping(index: int, node: dict):
    # print(node)
    if node["properties"]["label"] == "Part":
        return "green"
    elif node["properties"]["label"] == "Product":
        return "red"
    elif node["properties"]["label"] == "Assembly":
        return "orange"
    elif node["properties"]["label"] == "Specification":
        return "blue"
    elif node["properties"]["label"] == "Description":
        return "cyan"
    elif node["properties"]["label"] == "Supplier":
        return "pink"
    
def custom_node_label_mapping(index: int, node: dict):
    # print(node)
    if node["properties"]["label"] == "Part":
        return node["properties"]["sku"] + " - " + node["properties"]["name"]
    elif node["properties"]["label"] == "Product":
        return node["properties"]["sku"] + " - " + node["properties"]["name"]
    elif node["properties"]["label"] == "Assembly":
        return node["properties"]["sku"] + " - " + node["properties"]["name"]
    elif node["properties"]["label"] == "Specification":
        return "v: " + node["properties"]["version"]
    elif node["properties"]["label"] == "Description":
        return "v: " + node["properties"]["version"]
    elif node["properties"]["label"] == "Supplier":
        return node["properties"]["name"] + " - " + node["properties"]["country"]


# TODO

 - set weight of node based attribute, eg cost [Problem: cost lives on the relationship, not the node]

In [157]:
# Render ALL graph...

w = GraphWidget(graph=graph)
w.directed = False
w.set_graph_layout("orthogonal")
w.set_neighborhood(1,[graph.nodes.get(0).element_id])
w.set_sidebar(enabled = False, start_with = "Neighbourhood")
w.set_overview(False)

w.set_node_color_mapping(custom_node_color_mapping)
# w.set_node_color_mapping(lambda index, node : "blue" if node["properties"]["label"] == "Part" else ("orange" if node["properties"]["label"] == "SubAssembly" else "red"))
w.set_edge_color_mapping(lambda index, edge : "orange")
# w.set_node_label_mapping(lambda index, node : node["properties"]["sku"] + " - " + node["properties"]["name"])
w.set_node_label_mapping(custom_node_label_mapping)
# w.set_node_scale_factor_mapping(lambda index,node: node["properties"]["cost"] / 1000 if node["properties"]["label"] == "Part" else 1)

w.show()


GraphWidget(layout=Layout(height='500px', width='100%'))

# Product Cost

(Product) -> [Has] -> (Part) -> [Selected] -> (Supplier)

To find the product cost, we have to establish the

 - the parts that make up the product
 - the SELECTED suppliers
 - get the qty from both product->HAS->part and assembly->HAS->part relationships
 - get the cost on the SELECTED relationship
 - total the qty * cost


In [166]:
# Calculate Bike Cost

## Cost is on the (Part)-[SELECTED]->(Supplier)
## MATCH (p:Part)-[r:SELECTED]->(s:Supplier) RETURN SUM(r.cost) AS totalSum

cost = 0

with driver.session() as session:
    result = session.run("MATCH(product:Product)-[h1:HAS]-(ass:Assembly)-[h2:HAS]-(part:Part)-[s:SELECTED]-(supplier:Supplier) WHERE product.sku = 'ss01' return product.sku as sku, h1.qty as a_qty, h2.qty as p_qty, sum(s.cost) as cost;")
    # record = result.single()  # Assuming the query returns a single record

    for record in result.data():
        sku = record["sku"]
        cost += (record["a_qty"] * record["p_qty"] * record["cost"])

    print("Total Cost for the Product [" + sku + "]:", f"${cost/100:,.2f}")
    


Total Cost for the Product [ss01]: $112.72


In [160]:
# Show Supplier Cloud

with driver.session(database="neo4j") as session:
  graph = session.run("MATCH (p:Part)-[r:SELECTED]->(s:Supplier) RETURN p,r,s").graph()

w = GraphWidget(graph=graph)
w.directed = False
w.set_graph_layout("radial")
# w.set_neighborhood(1,[graph.nodes.get(0).element_id])
w.set_sidebar(enabled = False, start_with = "Neighbourhood")
w.set_overview(False)

w.set_node_color_mapping(custom_node_color_mapping)
# w.set_node_color_mapping(lambda index, node : "blue" if node["properties"]["label"] == "Part" else ("orange" if node["properties"]["label"] == "SubAssembly" else "red"))
w.set_edge_color_mapping(lambda index, edge : "orange")
# w.set_node_label_mapping(lambda index, node : node["properties"]["sku"] + " - " + node["properties"]["name"])
w.set_node_label_mapping(custom_node_label_mapping)
# w.set_node_scale_factor_mapping(lambda index,node: node["properties"]["cost"] / 1000 if node["properties"]["label"] == "Part" else 1)

w.show()


GraphWidget(layout=Layout(height='500px', width='100%'))

In [161]:
# Show eBOM

import pandas as pd

cypher_query = """
MATCH (product:Product)-[h1:HAS]-(assembly:Assembly)-[h2:HAS]-(part:Part)
WHERE product.sku = 'ss01'
RETURN part.sku as SKU, part.name AS Part, h1.qty as A_Qty, h2.qty as P_Qty
ORDER BY part.name
"""

with driver.session() as session:
    result = session.run(cypher_query)
    df = pd.DataFrame(result.data(), columns=result.keys())
    
df["Total"] = df["A_Qty"] * df["P_Qty"]

# Print the DataFrame as a table
print(df)

        SKU              Part  A_Qty  P_Qty  Total
0      ch01             Chain      1      1      1
1      cr01       Chain Rings      1      1      1
2      cs01           Cog set      1      1      1
3      fo01              Fork      1      1      1
4      fr01             Frame      1      1      1
5      fd01  Front Derailleur      1      1      1
6      fb01       Front brake      1      1      1
7   drk_grn             Green      1      2      2
8     hbr01         Handlebar      1      1      1
9     hdg01    Handlebar grip      1      2      2
10     hd01              Head      1      1      1
11     hb01               Hub      2      1      2
12     pd01             Pedal      1      2      2
13     rd01   Rear Derailluer      1      1      1
14     rb01       Rear brakes      1      1      1
15     rm01         Rim - STD      2      1      2
16      s01            Saddle      1      1      1
17     se02         Seat post      1      1      1
18     sh01           Shifter  

In [162]:
# Show mBOM

import pandas as pd

cypher_query = """
MATCH (product:Product)-[h1:HAS]-(assembly:Assembly)-[h2:HAS]-(part:Part)-[s:SELECTED]-(supplier:Supplier)
WHERE product.sku = 'ss01'
RETURN assembly.order AS a_order, assembly.name AS assembly, 
       h2.order AS p_order, part.name AS part, h2.qty as Qty, supplier.name AS supplier, s.cost as cost
ORDER BY assembly.order, h2.order
"""

cypher_query = """
MATCH (product:Product)-[h1:HAS]-(assembly:Assembly)-[h2:HAS]-(part:Part)-[s:SELECTED]-(supplier:Supplier)
WHERE product.sku = 'ss01'
RETURN assembly.order AS a_order, assembly.name AS assembly, 
       h2.order AS p_order, part.name AS part, h1.qty as A_Qty, h2.qty as P_Qty, s.cost as cost
ORDER BY assembly.order, h2.order
"""

with driver.session() as session:
    result = session.run(cypher_query)
    df = pd.DataFrame(result.data(), columns=result.keys())
    
df["total"] = df["A_Qty"] * df["P_Qty"] * df["cost"]

# Print the DataFrame as a table
print(df)

print()
print()

total = df['total'].sum()
print(total)
print("Total Cost for the Product [" + sku + "]:", f"${total/100:,.2f}")


    a_order        assembly  p_order              part  A_Qty  P_Qty  cost  \
0         1  Wheel Assembly        1               Hub      2      1    60   
1         1  Wheel Assembly        2       Spoke - STD      2     21   100   
2         1  Wheel Assembly        3         Rim - STD      2      1   600   
3         1  Wheel Assembly        4             Valve      2      1    42   
4         1  Wheel Assembly        5        Tire - STD      2      1   568   
5         2     Saddle Area        1         Seat post      1      1   222   
6         2     Saddle Area        2            Saddle      1      1   111   
7         3       Front Set        1         Handlebar      1      1   234   
8         3       Front Set        2    Handlebar grip      1      2   121   
9         3       Front Set        3       Front brake      1      1   451   
10        3       Front Set        4           Shifter      1      1   221   
11        3       Front Set        5              Fork      1   